In [15]:

from __future__ import print_function

from scipy.misc import imsave
import numpy as np
import time
from keras.applications import vgg19
from keras import backend as K

from keras.preprocessing import image
from keras.models import Model
from keras.applications.vgg19 import preprocess_input

import matplotlib.pyplot as plt

In [2]:
content = image.load_img("chien.jpg")
content_array = image.img_to_array(content)


height = content_array.shape[0]
width = content_array.shape[1]


style = image.load_img("style.jpg", target_size=(height,width))
style_array = image.img_to_array(style)

content_array = np.expand_dims(content_array, axis=0)
content_array = preprocess_input(content_array)
style_array = np.expand_dims(style_array, axis=0)
style_array = preprocess_input(style_array)

assert content_array.shape == style_array.shape

In [3]:
model = vgg19.VGG19(weights='imagenet', include_top=False)

In [4]:
layers_name = [layer.name for layer in model.layers]

In [5]:

model_dict = dict([(name,Model(inputs=model.input, outputs=model.get_layer(name).output) ) for name in layers_name])






In [6]:
'''
layer = model_dict["block1_conv1"]


content2 = image.load_img("chien.jpg",target_size=(224,224))
content2_array = image.img_to_array(content2)
content2_array = np.expand_dims(content2_array, axis=0)
#content2_array = preprocess_input(content2_array)
model2 = vgg19.VGG19(weights='imagenet', include_top=True)
preds = model2.predict(content2_array)
decode_predictions(preds)
'''



'\nlayer = model_dict["block1_conv1"]\n\n\ncontent2 = image.load_img("chien.jpg",target_size=(224,224))\ncontent2_array = image.img_to_array(content2)\ncontent2_array = np.expand_dims(content2_array, axis=0)\n#content2_array = preprocess_input(content2_array)\nmodel2 = vgg19.VGG19(weights=\'imagenet\', include_top=True)\npreds = model2.predict(content2_array)\ndecode_predictions(preds)\n'

In [7]:
filter_style = {}
filter_content = {}

layers_style = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1", "block5_conv1"]
layers_content = ["block4_conv1"]

for name in layers_name:
    print("Layer name : ", name)
    if name in layers_style:
        model_layer = model_dict[name]
        conv = model_layer.predict(style_array)
        filter_style[name] = conv
        print("Adding filter of shape ", conv.shape, " to style list")
    
    if name in layers_content:
        model_layer = model_dict[name]
        conv = model_layer.predict(style_array)
        filter_content[name] = conv
        print("Adding filter of shape ", conv.shape, " to content list")
    

Layer name :  input_1
Layer name :  block1_conv1
Adding filter of shape  (1, 467, 700, 64)  to style list
Layer name :  block1_conv2
Layer name :  block1_pool
Layer name :  block2_conv1
Adding filter of shape  (1, 233, 350, 128)  to style list
Layer name :  block2_conv2
Layer name :  block2_pool
Layer name :  block3_conv1
Adding filter of shape  (1, 116, 175, 256)  to style list
Layer name :  block3_conv2
Layer name :  block3_conv3
Layer name :  block3_conv4
Layer name :  block3_pool
Layer name :  block4_conv1
Adding filter of shape  (1, 58, 87, 512)  to style list
Adding filter of shape  (1, 58, 87, 512)  to content list
Layer name :  block4_conv2
Layer name :  block4_conv3
Layer name :  block4_conv4
Layer name :  block4_pool
Layer name :  block5_conv1
Adding filter of shape  (1, 29, 43, 512)  to style list
Layer name :  block5_conv2
Layer name :  block5_conv3
Layer name :  block5_conv4
Layer name :  block5_pool


In [10]:
gram_dict_style = {}
for key in filter_style:
    print(key)
    filters= filter_style[key]
    shape = filters.shape    
    print(shape)
    gram = np.zeros((shape[3],shape[3]))
    
    for i in range(shape[3]):
        for j in range(shape[3]):
            #print(i,j)
            response_i = filters[0,:,:,i].ravel()
            response_j = filters[0,:,:,j].ravel()
            
            corr = response_i.dot(response_j)
            gram[i,j] = corr
    
    gram_dict_style[key] = gram

block3_conv1
(1, 116, 175, 256)
block2_conv1
(1, 233, 350, 128)
block4_conv1
(1, 58, 87, 512)
block5_conv1
(1, 29, 43, 512)
block1_conv1
(1, 467, 700, 64)


In [28]:
input_img = model.input

img_data = np.random.random((1, height, width,3)) * 255

img_data = preprocess_input(img_data)


In [35]:
filter_style_img = {}
filter_content_img = {}

for name in layers_name:
    print("Layer name : ", name)
    if name in layers_style:
        model_layer = model_dict[name]
        conv = model_layer.output
        ?conv
        filter_style_img[name] = conv
        print("Adding model to style list")
    
    if name in layers_content:
        model_layer = model_dict[name]
        conv = model_layer.output
        filter_content_img[name] = conv
        print("Adding filter to content list")
        

gram_dict_style_img = {}
for key in filter_style_img:
    print(key)
    filters= filter_style_img[key]
    shape = filters.get_shape()    
    print(shape)
    gram= np.zeros((shape[3],shape[3]))
    
    for i in range(shape[3]):
        for j in range(shape[3]):
            #print(i,j)
            response_i = filters[0,:,:,i].ravel()
            response_j = filters[0,:,:,j].ravel()
            
            corr = response_i.dot(response_j)
            gram[i,j] = corr
    
    gram_dict_style_img[key] = gram

Layer name :  input_1
Layer name :  block1_conv1
Adding model to style list
Layer name :  block1_conv2
Layer name :  block1_pool
Layer name :  block2_conv1
Adding model to style list
Layer name :  block2_conv2
Layer name :  block2_pool
Layer name :  block3_conv1
Adding model to style list
Layer name :  block3_conv2
Layer name :  block3_conv3
Layer name :  block3_conv4
Layer name :  block3_pool
Layer name :  block4_conv1
Adding model to style list
Adding filter to content list
Layer name :  block4_conv2
Layer name :  block4_conv3
Layer name :  block4_conv4
Layer name :  block4_pool
Layer name :  block5_conv1
Adding model to style list
Layer name :  block5_conv2
Layer name :  block5_conv3
Layer name :  block5_conv4
Layer name :  block5_pool
block3_conv1
(?, ?, ?, 256)


AttributeError: 'Tensor' object has no attribute 'ravel'

In [37]:
?K.mean

In [27]:

print(img_data.shape)
print(img_data[0].shape)
plt.imshow(img_data[0])
plt.show()

(1, 467, 700, 3)
(467, 700, 3)


In [9]:
a = np.array([[1,1,1],[2,2,2]])
ar = a.ravel()
art = a.ravel().T

print(ar)
print(art)
print(ar.dot(ar))
print(ar.dot(art))

[1 1 1 2 2 2]
[1 1 1 2 2 2]
15
15
